# 类继承关系
```mermaid
classDiagram
    class SafeToStr
    class Sub
    class Rep
    class RepEval
    class RepFunc
    SafeToStr <|-- Sub
    SafeToStr <|-- Rep
    SafeToStr <|-- RepEval
    SafeToStr <|-- RepFunc
```

In [10]:
from vectorbt.utils import *

# class Sub(SafeToStr)
字符串模板替换类。支持延迟求值以及映射合并。
```python
class Sub(SafeToStr):
    def __init__(self, template: tp.Union[str, Template], mapping: tp.Optional[tp.Mapping] = None) -> None:
        self._template = template  # 存储原始模板，可以是字符串或Template对象
        self._mapping = mapping    # 存储初始映射字典

    @property
    def template(self) -> Template:
        if not isinstance(self._template, Template):
            return Template(self._template)
        return self._template

    @property
    def mapping(self) -> tp.Mapping:
        if self._mapping is None:
            return {}
        return self._mapping

    def substitute(self, mapping: tp.Optional[tp.Mapping] = None) -> str:
        mapping = merge_dicts(self.mapping, mapping)
        return self.template.substitute(mapping)

    def __str__(self) -> str:
        return f"{self.__class__.__name__}(" \
               f"template=\"{self.template.template}\", " \
               f"mapping={prepare_for_doc(self.mapping)})"
```

In [3]:
sub = Sub('$greeting $name!', {'greeting': 'Hello'})
result1 = sub.substitute({'name': 'Bob', 'greeting': 'Hi'})  # Hi Bob!
result2 = sub.substitute({'name': 'Charlie'})  # Hello Charlie!
print(result1)
print(result2)

Hi Bob!
Hello Charlie!


# class Rep(SafeToStr)
简单键值替换类。
```python
class Rep(SafeToStr):
    def __init__(self, key: tp.Hashable, mapping: tp.Optional[tp.Mapping] = None) -> None:
        self._key = key        # 存储要查找的键名
        self._mapping = mapping # 存储初始映射字典

    @property
    def key(self) -> tp.Hashable:
        return self._key

    @property
    def mapping(self) -> tp.Mapping:
        if self._mapping is None:
            return {} 
        return self._mapping

    def replace(self, mapping: tp.Optional[tp.Mapping] = None) -> tp.Any:
        mapping = merge_dicts(self.mapping, mapping)
        return mapping[self.key]

    def __str__(self) -> str:
        return f"{self.__class__.__name__}(" \
               f"key='{self.key}', " \
               f"mapping={prepare_for_doc(self.mapping)})"
```

In [4]:
rep = Rep('config', {'config': 'default'})
result = rep.replace({'config': 'production'})
print(result)

production


# class RepEval(SafeToStr)
表达式求值替换类。
```python
class RepEval(SafeToStr):
    def __init__(self, expression: str, mapping: tp.Optional[tp.Mapping] = None) -> None:
        self._expression = expression  # 存储要求值的表达式字符串
        self._mapping = mapping        # 存储初始映射字典

    @property
    def expression(self) -> str:
        return self._expression

    @property
    def mapping(self) -> tp.Mapping:
        if self._mapping is None:
            return {}
        return self._mapping

    def eval(self, mapping: tp.Optional[tp.Mapping] = None) -> tp.Any:
        mapping = merge_dicts(self.mapping, mapping)  # 合并初始映射和当前映射
        return eval(self.expression, {}, mapping)     # 使用空的全局环境和合并后的局部环境求值

    def __str__(self) -> str:
        return f"{self.__class__.__name__}(" \
               f"expression=\"{self.expression}\", " \
               f"mapping={prepare_for_doc(self.mapping)})"
```

In [5]:
rep_eval = RepEval('x + y * 2', {'x': 10, 'y': 5})
result = rep_eval.eval()
print(result)  # 20

rep_eval = RepEval('max_val if use_max else min_val')
result = rep_eval.eval({'max_val': 100, 'min_val': 1, 'use_max': True})
print(result)  # 100

rep_eval = RepEval('[x*2 for x in numbers if x > 0]')
result = rep_eval.eval({'numbers': [-1, 2, -3, 4]})
print(result)  # [4, 8]

20
100
[4, 8]


# class RepFunc(SafeToStr)
函数调用替换类
```python
class RepFunc(SafeToStr):
    def __init__(self, func: tp.Callable, mapping: tp.Optional[tp.Mapping] = None) -> None:
        self._func = func       # 存储要调用的函数对象
        self._mapping = mapping # 存储初始映射字典

    @property
    def func(self) -> tp.Callable:
        return self._func

    @property
    def mapping(self) -> tp.Mapping:
        if self._mapping is None:
            return {}
        return self._mapping

    def call(self, mapping: tp.Optional[tp.Mapping] = None) -> tp.Any:
        mapping = merge_dicts(self.mapping, mapping)  # 合并初始映射和当前映射
        func_arg_names = get_func_arg_names(self.func)  # 获取函数的参数名列表
        func_kwargs = dict()                          # 创建空的关键字参数字典
        for k, v in mapping.items():                  # 遍历合并后的映射
            if k in func_arg_names:                   # 检查键是否是函数的参数名
                func_kwargs[k] = v                    # 如果是，则添加到函数参数字典中
        return self.func(**func_kwargs)               # 使用提取的参数调用函数

    def __str__(self) -> str:
        return f"{self.__class__.__name__}(" \
               f"func={self.func}, " \
               f"mapping={prepare_for_doc(self.mapping)})"
```

In [7]:
def calculate(x, y, operation='add'):
    if operation == 'add':
        return x + y
    elif operation == 'multiply':
        return x * y

rep_func = RepFunc(calculate, {'x': 10, 'y': 5})
result = rep_func.call({'operation': 'multiply'})
print(result)  # 50

50


# def has_templates(obj: tp.Any)
检查对象是否包含模板元素
```python
def has_templates(obj: tp.Any) -> tp.Any:
    if isinstance(obj, RepFunc):
        return True
    if isinstance(obj, RepEval):
        return True
    if isinstance(obj, Rep):
        return True
    if isinstance(obj, Sub):
        return True
    if isinstance(obj, Template):
        return True
    if isinstance(obj, dict):
        for k, v in obj.items():
            if has_templates(v):
                return True
    if isinstance(obj, (tuple, list, set, frozenset)):
        for v in obj:
            if has_templates(v):
                return True
    return False
```

# def deep_substitute
深度递归 `obj`，找到其中的模板对象并进行替换
```python
def deep_substitute(obj: tp.Any,
                    mapping: tp.Optional[tp.Mapping] = None,
                    safe: bool = False,
                    make_copy: bool = True) -> tp.Any:
    """
    Args:
        obj: 要处理的对象，可以包含模板元素的任意数据结构
        mapping: 可选的映射字典，提供模板替换所需的值
        safe: 安全模式标志，为True时遇到错误不抛出异常而是返回原模板
        make_copy: 是否创建对象副本，为True时会复制容器对象以避免修改原对象
        
    Returns:
        Any: 替换后的对象，如果没有模板元素则返回原对象
    """
    if mapping is None:
        mapping = {}
    if not has_templates(obj):
        return obj
    try:
        if isinstance(obj, RepFunc):  
            return obj.call(mapping)
        if isinstance(obj, RepEval):
            return obj.eval(mapping)
        if isinstance(obj, Rep):
            return obj.replace(mapping) 
        if isinstance(obj, Sub):
            return obj.substitute(mapping)
        if isinstance(obj, Template):
            return obj.substitute(mapping)
        if isinstance(obj, dict):                       # 如果是字典类型
            if make_copy:
                obj = copy(obj)                         # 创建字典的浅复制
            for k, v in obj.items():
                set_dict_item(obj, k, deep_substitute(v, mapping=mapping, safe=safe), force=True)  # 递归替换值并强制设置
            return obj
        if isinstance(obj, list):                       # 如果是列表类型
            if make_copy:
                obj = copy(obj)                         # 创建列表的浅复制
            for i in range(len(obj)):
                obj[i] = deep_substitute(obj[i], mapping=mapping, safe=safe)  # 递归替换每个元素
            return obj
        if isinstance(obj, (tuple, set, frozenset)): 
            result = [] 
            for o in obj:
                result.append(deep_substitute(o, mapping=mapping, safe=safe))  # 递归替换每个元素并添加到结果列表
            if checks.is_namedtuple(obj):               # 检查是否是命名元组
                return type(obj)(*result)               # 如果是命名元组，使用原类型和结果重新构造
            return type(obj)(result)                    # 否则使用原类型构造新的容器对象
    except Exception as e:
        if not safe:                                    # 如果不是安全模式
            raise e                                     # 重新抛出异常
    return obj                                          # 如果是安全模式或没有模板，返回原对象
```

In [12]:
data = {
    'config': Rep('env'),
    'values': [Sub('${prefix}_${suffix}'), Rep('count')],  # 使用花括号
    'meta': {'template': RepEval('x * 2')}
}
mapping = {'env': 'prod', 'prefix': 'data', 'suffix': 'file', 'count': 42, 'x': 21}
result = deep_substitute(data, mapping)
print(result)

{'config': 'prod', 'values': ['data_file', 42], 'meta': {'template': 42}}
